[This](https://github.com/pgmpy/pgmpy_notebook/blob/master/notebooks/9.%20Learning%20Bayesian%20Networks%20from%20Data.ipynb) shows how to infer a bn from data

In [3]:
import pgmpy

In [4]:
import pandas as pd
import numpy as np

In [5]:
cars_data = pd.read_csv('data/car.data', names=["Buying", "Maintenance", "Doors", "People", "LugBoot", "Safety", "Acceptability"])
cars_data

,Buying,Maintenance,Doors,People,LugBoot,Safety,Acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [6]:
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BicScore

hc = HillClimbSearch(cars_data, scoring_method=BicScore(cars_data))
best_model = hc.estimate()
print(best_model.edges())

  0%|          | 9/1000000 [00:00<15:14:36, 18.22it/s]


[('Buying', 'Maintenance'), ('Safety', 'People'), ('Safety', 'LugBoot'), ('Acceptability', 'Safety'), ('Acceptability', 'People'), ('Acceptability', 'Buying'), ('Acceptability', 'Maintenance'), ('Acceptability', 'LugBoot')]


In [7]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator

bayesian_model = BayesianModel(best_model.edges())
bayesian_model.fit(cars_data, estimator=MaximumLikelihoodEstimator)
for cpd in bayesian_model.get_cpds():
    print(cpd)

+----------------------+-----------+
| Acceptability(acc)   | 0.222222  |
+----------------------+-----------+
| Acceptability(good)  | 0.0399306 |
+----------------------+-----------+
| Acceptability(unacc) | 0.700231  |
+----------------------+-----------+
| Acceptability(vgood) | 0.0376157 |
+----------------------+-----------+
+---------------+---------------------+---------------------+----------------------+----------------------+
| Acceptability | Acceptability(acc)  | Acceptability(good) | Acceptability(unacc) | Acceptability(vgood) |
+---------------+---------------------+---------------------+----------------------+----------------------+
| Buying(high)  | 0.28125             | 0.0                 | 0.26776859504132233  | 0.0                  |
+---------------+---------------------+---------------------+----------------------+----------------------+
| Buying(low)   | 0.23177083333333334 | 0.6666666666666666  | 0.21322314049586777  | 0.6                  |
+---------------+--

In [8]:
from pgmpy.inference import VariableElimination

exact_inference = VariableElimination(bayesian_model)

Now, it would be interesting to answer some questions:
* Is maintenance harder for 2-seaters than for family cars?

In [9]:
print(exact_inference.query(["Maintenance"],{'People':"2"}))
print(exact_inference.query(["Maintenance"],{'People':"4"}))



Eliminating: Acceptability: 100%|██████████| 4/4 [00:00<00:00, 460.98it/s]

Finding Elimination Order: :   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Eliminating: LugBoot:   0%|          | 0/4 [00:00<?, ?it/s]

Eliminating: Buying:   0%|          | 0/4 [00:00<?, ?it/s] 

Eliminating: Safety:   0%|          | 0/4 [00:00<?, ?it/s]

Eliminating: Acceptability: 100%|██████████| 4/4 [00:00<00:00, 535.62it/s]


+--------------------+--------------------+
| Maintenance        |   phi(Maintenance) |
+====================+====================+
| Maintenance(high)  |             0.2595 |
+--------------------+--------------------+
| Maintenance(low)   |             0.2215 |
+--------------------+--------------------+
| Maintenance(med)   |             0.2215 |
+--------------------+--------------------+
| Maintenance(vhigh) |             0.2975 |
+--------------------+--------------------+
+--------------------+--------------------+
| Maintenance        |   phi(Maintenance) |
+====================+====================+
| Maintenance(high)  |             0.2450 |
+--------------------+--------------------+
| Maintenance(low)   |             0.2648 |
+--------------------+--------------------+
| Maintenance(med)   |             0.2646 |
+--------------------+--------------------+
| Maintenance(vhigh) |             0.2256 |
+--------------------+--------------------+


It looks like it is, indeed. Then,
* Are cars with high maintenance safer?

In [10]:
print(exact_inference.query(["Safety"],{'Maintenance':"vhigh"}))



  0%|          | 0/4 [00:00<?, ?it/s]

Finding Elimination Order: :   0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


Eliminating: LugBoot:   0%|          | 0/4 [00:00<?, ?it/s]


Finding Elimination Order: : 100%|██████████| 4/4 [00:00<00:00, 545.85it/s]



Eliminating: Buying:   0%|          | 0/4 [00:00<?, ?it/s]


Eliminating: Acceptability: 100%|██████████| 4/4 [00:00<00:00, 508.65it/s]


+--------------+---------------+
| Safety       |   phi(Safety) |
+==============+===============+
| Safety(high) |        0.2793 |
+--------------+---------------+
| Safety(low)  |        0.3967 |
+--------------+---------------+
| Safety(med)  |        0.3240 |
+--------------+---------------+


Looks like they're not. 
* Is the lugboot involved in safety?

In [11]:
print(exact_inference.query(["LugBoot"],{'Safety':"low"}))

Eliminating: Acceptability: 100%|██████████| 4/4 [00:00<00:00, 456.52it/s]


+----------------+----------------+
| LugBoot        |   phi(LugBoot) |
+================+================+
| LugBoot(big)   |         0.3333 |
+----------------+----------------+
| LugBoot(med)   |         0.3333 |
+----------------+----------------+
| LugBoot(small) |         0.3333 |
+----------------+----------------+


## Approximate inference
We'd now like to try and infer probabilities from samples, by extracting $n$ samples from the distribution and calculating the probability.
According to **Hoeffding’s inequality**, we know that
$$
P(|s-p|>\epsilon) \le 2 e^{-2n\epsilon^2}
$$
which we can use to get a desired number of samples to extract.
We'll try two different techniques, **Rejection Sampling** and **Likelihood weighting**. 
The first estimates the probability by counting the samples that are consistent with the evidence, and it is a process similar to what we do in the real world.
The latter, instead, only samples nonevidence variables, then weights them by the likelihood they accord the evidence.

In [16]:
class SampleTester():
    def __init__(self, bayesian_model):
        self.bayesian_model = bayesian_model
        self.sampler = BayesianModelSampling(bayesian_model)
        self.exact_inference = VariableElimination(bayesian_model)
    def test_sample_methods(self, evidence, query, sample_size_min = 100, sample_size_max=10**6, num_of_experiments=20):
        test_range = range(sample_size_min, sample_size_max, int((sample_size_max-sample_size_min)/num_of_experiments))
        errors = {}
        for size in test_range:
            error = self.process_lws(size, evidence, query)
            errors[size]= error
        print(f"Experiment finished with achieved errors: {errors}")
    def process_lws(self, size, evidence, query):
        likelihood_sample = self.sampler.likelihood_weighted_sample(evidence=evidence, size=size, return_type='recarray')
        sample_probs = self.return_weighted_probs(likelihood_sample[query],likelihood_sample['_weight'])
        exact_result = self.exact_inference.query([query], dict(evidence)).values
        absolute_error = self.calculate_error(sample_probs, exact_result)
        return absolute_error

    def process_rs(self, size, evidence, query):
        rejection_sample = self.sampler.rejection_sample(evidence=evidence, size=size, return_type='recarray')[query]
        sample_probs = self.return_probs(rejection_sample)
        exact_result = self.exact_inference.query([query], dict(evidence)).values
        absolute_error = self.calculate_error(sample_probs, exact_result)
        return absolute_error
    def calculate_error(self, sample_probs, exact_probs):
        absolute_error = np.fromiter(sample_probs.values(), dtype=float) - np.array(exact_probs)
        return absolute_error
    def return_probs(self, samples):
        unique, counts = np.unique(samples, return_counts=True) # Get the unique values and their counts
        counts = (counts/len(samples))# Divide the counts by the total number, getting a probability from 0 to 1
        return dict(zip(unique, counts)) # Zip the value and its probability in a dict
    def return_weighted_probs(self, samples, weights):
        unique = np.unique(samples)
        counts = np.zeros(len(np.unique(samples))) # We fill a zero array for the weights sum, the  we'll divide it by the sum of weights
        iterator = np.nditer(samples, flags=['f_index'])
        for value in iterator:
            counts[value] += weights[iterator.index]
        counts = (counts/np.sum(weights))
        return dict(zip(unique, counts)) # Zip the value and its probability in a dict

from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import State


evidence = [State('Safety', 'high'),State('Maintenance', 'low')]

st = SampleTester(bayesian_model)
st.test_sample_methods(evidence, "Acceptability", num_of_experiments=4)

Eliminating: Buying: 100%|██████████| 3/3 [00:00<00:00, 522.50it/s]

Finding Elimination Order: :   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Eliminating: LugBoot:   0%|          | 0/3 [00:00<?, ?it/s]

Eliminating: People:   0%|          | 0/3 [00:00<?, ?it/s] 

Eliminating: Buying: 100%|██████████| 3/3 [00:00<00:00, 402.19it/s]


  0%|          | 0/3 [00:00<?, ?it/s]

Finding Elimination Order: :   0%|          | 0/3 [00:00<?, ?it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


Eliminating: LugBoot:   0%|          | 0/3 [00:00<?, ?it/s]


Eliminating: People:   0%|          | 0/3 [00:00<?, ?it/s] 


Eliminating: Buying: 100%|██████████| 3/3 [00:00<00:00, 524.31it/s]



  0%|          | 0/3 [00:00<?, ?it/s]


Finding Elimination Order: :   0%|          | 0/3 [00:00<?, ?it/s]



  0%|          | 0/3 [00:00<?, ?it/s]



Eliminating: LugBoot:   0%|          | 0/3 [00:00<?, ?it/s]



Eliminating: People:   0%|          | 0/3 [00:00<?, ?it/s] 



Eliminating:

Experiment finished with achieved errors: {100: array([ 0.01611146, -0.04672223,  0.09315625, -0.06254549]), 250075: array([ 0.00150097,  0.00031586, -0.00228624,  0.00046942]), 500050: array([ 0.00100012,  0.00127742,  0.00091852, -0.00319606]), 750025: array([ 0.00060661, -0.0002194 , -0.00028043, -0.00010678])}
